Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [ ]:
fake_news_dataset = pd.read_csv('/content/Fake.csv')
true_news_dataset = pd.read_csv('/content/true.csv')
fake_news_dataset.shape
true_news_dataset.shape

NameError: name 'pd' is not defined

In [ ]:
fake_news_dataset.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
true_news_dataset.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fake_news_dataset.tail()

,title,text,subject,date
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016"


In [ ]:
true_news_dataset['isfake'] = 0
fake_news_dataset['isfake'] = 1

In [ ]:
fake_news_dataset.head()

,title,text,subject,date,isfake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
true_news_dataset.head()

,title,text,subject,date,isfake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [ ]:
merged_dataset = pd.concat([fake_news_dataset,true_news_dataset])

In [ ]:
merged_dataset.head()

,title,text,subject,date,isfake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
merged_dataset.tail()

,title,text,subject,date,isfake
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",0


In [ ]:
merged_dataset = merged_dataset.sample(frac = 1).reset_index(drop= True)

In [ ]:
merged_dataset.head()

,title,text,subject,date,isfake
0,Good Riddance: James Clapper Resigns as Direct...,21st Century Wire says Like CIA Director John ...,Middle-east,"November 19, 2016",1
1,Factbox: Big-ticket items at center of Congres...,(Reuters) - The U.S. Congress on Thursday appr...,politicsNews,"December 21, 2017",0
2,FLASHBACK To Time When ABC News’ #LyinBrianRos...,"As the name of the suspect, identified as 24-y...",left-news,"Dec 3, 2017",1
3,EPIC! CHECK OUT THE T-SHIRTS Two Guys Wore Beh...,This is really epic and just hysterical! It s ...,politics,"Feb 9, 2016",1
4,UK minister says U.S. will drive hard bargain ...,LONDON (Reuters) - Foreign minister Boris John...,worldnews,"November 1, 2017",0


In [ ]:
merged_dataset.tail()

,title,text,subject,date,isfake
44893,WATCH: AMAZING THING HAPPENED When H.S. Valedi...,A 70-year tradition at East Liverpool High Sch...,left-news,"Jun 27, 2017",1
44894,‘Stand Your Ground’: FL Man Shoots At Teens P...,Florida s toxic and dangerous stand your grou...,News,"July 17, 2016",1
44895,Conservatives Will Hate Why Obama Invited Thi...,President Obama is using his final State of th...,News,"January 11, 2016",1
44896,LIBERAL DUMMY GARY JOHNSON Can’t Name One Fore...,Maybe comedy is a better career choice for Gar...,left-news,"Sep 29, 2016",1
44897,"House GOP Holds Zika Bill Hostage, Proposes P...",Pregnant women throughout the America s are th...,News,"July 7, 2016",1


In [ ]:
merged_dataset.tail(10)

,title,text,subject,date,isfake
44888,Factbox: Trump on Twitter (Jan 19)-Rev. Graham...,The following bullet points are from the U.S. ...,politicsNews,"January 20, 2017",0
44889,"President Trump Fires Acting Attorney General,...",21st Century Wire says In a press release from...,Middle-east,"January 31, 2017",1
44890,WATCH THE CRIMINAL HISTORY OF HILLARY AND BILL...,Everything you ever wanted to know about Hilla...,left-news,"Mar 17, 2016",1
44891,Austria election victor calls for end to Turke...,BRUSSELS (Reuters) - Austria s likely next cha...,worldnews,"October 19, 2017",0
44892,FEDS SPEND BIG BUCKS On Minority-Only Obesity ...,The Obama administration has spent over $166 m...,Government News,"Jun 1, 2016",1
44893,WATCH: AMAZING THING HAPPENED When H.S. Valedi...,A 70-year tradition at East Liverpool High Sch...,left-news,"Jun 27, 2017",1
44894,‘Stand Your Ground’: FL Man Shoots At Teens P...,Florida s toxic and dangerous stand your grou...,News,"July 17, 2016",1
44895,Conservatives Will Hate Why Obama Invited Thi...,President Obama is using his final State of th...,News,"January 11, 2016",1
44896,LIBERAL DUMMY GARY JOHNSON Can’t Name One Fore...,Maybe comedy is a better career choice for Gar...,left-news,"Sep 29, 2016",1
44897,"House GOP Holds Zika Bill Hostage, Proposes P...",Pregnant women throughout the America s are th...,News,"July 7, 2016",1


In [ ]:
merged_dataset.tail(20)

,title,text,subject,date,isfake
44878,Winning Democrat nomination needs voter coalit...,NEW YORK (Reuters) - The campaign of Democrati...,politicsNews,"March 2, 2016",0
44879,France's Macron says euro zone needs its own b...,PARIS (Reuters) - French President Emmanuel Ma...,worldnews,"September 26, 2017",0
44880,Russia's Lavrov tells Tillerson important not ...,MOSCOW (Reuters) - Russia’s Foreign Minister S...,politicsNews,"April 12, 2017",0
44881,HOW SO? CNN HOST Makes Claim President Trump I...,CNN anchor Brian Stelter went off on President...,politics,"Nov 19, 2017",1
44882,WATCH: GOP Senate Candidate Kept Referring To...,The presumed nomination of Donald Trump has gi...,News,"May 15, 2016",1
44883,LIBERAL TREVOR NOAH Didn’t Count On Conservati...,,left-news,"Dec 2, 2016",1
44884,HORRIFIC Human Trafficking Case: 8 PEOPLE FOUN...,Anyone who takes part in this type of sub-huma...,left-news,"Jul 23, 2017",1
44885,FBI Clears Michael Flynn Of Wrongdoing As Tru...,"On Tuesday, it was announced that Donald Trump...",News,"January 24, 2017",1
44886,POLICE IN GERMANY BEGIN RAIDS On Homes Of Face...,If Facebook has aligned themselves with German...,politics,"Jul 15, 2016",1
44887,South Dakota House fails to override veto of t...,(Reuters) - The South Dakota House of Represen...,politicsNews,"March 3, 2016",0


In [ ]:
merged_dataset = merged_dataset.drop(columns=['subject','date'],axis=1)

In [ ]:
merged_dataset.head()

,title,text,isfake
0,Good Riddance: James Clapper Resigns as Direct...,21st Century Wire says Like CIA Director John ...,1
1,Factbox: Big-ticket items at center of Congres...,(Reuters) - The U.S. Congress on Thursday appr...,0
2,FLASHBACK To Time When ABC News’ #LyinBrianRos...,"As the name of the suspect, identified as 24-y...",1
3,EPIC! CHECK OUT THE T-SHIRTS Two Guys Wore Beh...,This is really epic and just hysterical! It s ...,1
4,UK minister says U.S. will drive hard bargain ...,LONDON (Reuters) - Foreign minister Boris John...,0


In [ ]:
merged_dataset['content'] = merged_dataset['title'] + ' ' + merged_dataset['text']

In [ ]:
merged_dataset.head()

,title,text,isfake,content
0,Good Riddance: James Clapper Resigns as Direct...,21st Century Wire says Like CIA Director John ...,1,Good Riddance: James Clapper Resigns as Direct...
1,Factbox: Big-ticket items at center of Congres...,(Reuters) - The U.S. Congress on Thursday appr...,0,Factbox: Big-ticket items at center of Congres...
2,FLASHBACK To Time When ABC News’ #LyinBrianRos...,"As the name of the suspect, identified as 24-y...",1,FLASHBACK To Time When ABC News’ #LyinBrianRos...
3,EPIC! CHECK OUT THE T-SHIRTS Two Guys Wore Beh...,This is really epic and just hysterical! It s ...,1,EPIC! CHECK OUT THE T-SHIRTS Two Guys Wore Beh...
4,UK minister says U.S. will drive hard bargain ...,LONDON (Reuters) - Foreign minister Boris John...,0,UK minister says U.S. will drive hard bargain ...


In [ ]:
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# for nltk.pos_tag function
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to the format accepted by the lemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
import string
def data_preprocessing(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  words = word_tokenize(text)
  stop_words = stopwords.words('english')
  words = [word for word in words if word not in stop_words]
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(word,pos=get_wordnet_pos(word)) for word in words]
  text = ' '.join(words)
  return text

In [ ]:
#to reduce the time of execution small change in code we downloaded the pprocessed text data
merged_dataset['content'] = merged_dataset['content'].apply(data_preprocessing)
#processed_dataset = pd.read_csv('/content/processed_dataset.csv')
#merged_dataset['content'] = processed_dataset['content']
#preprocessed_dataset.head()

In [ ]:
merged_dataset.head()

,title,text,isfake,content
0,Good Riddance: James Clapper Resigns as Direct...,21st Century Wire says Like CIA Director John ...,1,good riddance james clapper resigns director u...
1,Factbox: Big-ticket items at center of Congres...,(Reuters) - The U.S. Congress on Thursday appr...,0,factbox bigticket item center congress spendin...
2,FLASHBACK To Time When ABC News’ #LyinBrianRos...,"As the name of the suspect, identified as 24-y...",1,flashback time abc news ’ lyinbrianross try ti...
3,EPIC! CHECK OUT THE T-SHIRTS Two Guys Wore Beh...,This is really epic and just hysterical! It s ...,1,epic check tshirts two guy wore behind hillary...
4,UK minister says U.S. will drive hard bargain ...,LONDON (Reuters) - Foreign minister Boris John...,0,uk minister say u drive hard bargain postbrexi...


In [ ]:
merged_dataset1 = merged_dataset

In [ ]:
import pandas as pd

# Assuming your DataFrame is named df
merged_dataset1.to_csv('processed_dataset.csv', index=False)

In [ ]:
from google.colab import files
files.download('processed_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(merged_dataset['content'], merged_dataset['isfake'], test_size=0.3,random_state=42)

In [ ]:
print(X_train.shape,X_test.shape)

(31428,) (13470,)


In [ ]:
countvectorizer = CountVectorizer()
term_frequency_matrix = countvectorizer.fit_transform(X_train)
tfidf = TfidfTransformer(norm = 'l2')
X_train = tfidf.fit_transform(term_frequency_matrix)

In [ ]:
import pickle
filename1 = 'tfidfmodel.sav'
filename2 = 'countvectmodel.sav'
pickle.dump(tfidf,open(filename1,'wb'))
pickle.dump(countvectorizer,open(filename2,'wb'))

In [ ]:
X_train

<31428x154758 sparse matrix of type '<class 'numpy.float64'>'
	with 4836288 stored elements in Compressed Sparse Row format>

**MODEL SELECTION**

In [ ]:
param_dist = {
    'C': np.logspace(-4, 4, 20),  # Regularization parameter
    'penalty': ['l1', 'l2'],       # Regularization type
    'solver': ['liblinear', 'saga']  # Solver algorithm
}


In [ ]:
X = merged_dataset['content']
y = merged_dataset['isfake']

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
log_reg = LogisticRegression()

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_dist,
    n_iter=80,  # Number of parameter settings to try
    cv=5,        # Number of cross-validation folds
    verbose=1,   # Print progress messages
    random_state=42,  # Seed for reproducibility
    n_jobs=-1    # Use all available cores
)
random_search.fit(X_train,y_train)

In [ ]:
best_params = random_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 78.47599703514607}

In [ ]:
logistic_regression_model = LogisticRegression(solver='liblinear', penalty='l1', C=78.47599703514607)
logistic_regression_model.fit(X_train,y_train)

LogisticRegression(C=78.47599703514607, penalty='l1', solver='liblinear')

In [ ]:
print('Accuracy score of the training data : ', accuracy_score(y_train, logistic_regression_model.predict(X_train)))

Accuracy score of the training data :  1.0


In [ ]:
print('Accuracy score of the test data : ', accuracy_score(y_test, logistic_regression_model.predict(tfidf.transform(countvectorizer.transform(X_test)))))

Accuracy score of the test data :  0.9970304380103935


In [ ]:
from sklearn.metrics import classification_report
y_pred = logistic_regression_model.predict(tfidf.transform(countvectorizer.transform(X_test)))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6465
           1       1.00      1.00      1.00      7005

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[6450   15]
 [  25 6980]]


In [ ]:
def predict(title,text):
  text = title + ' ' + text
  text = data_preprocessing(text)
  text = tfidf.transform(countvectorizer.transform([text]))
  result = logistic_regression_model.predict(text)[0]
  if result == 1:
    return 'Fake News'
  else:
    return 'True News'

In [ ]:
title = "President Trump Just Gave A HUGE Gift to #HurricaneHarvey Reliefâ€¦Walking the Walk!"
text =  'Yea Baby! President Trump walks the walk!President Trump just put his money where his mouth is! He donated $1 million to the Harvey Hurricane relief! What a great thing to do! He has a huge heart!Is this empathetic enough for the left? The media decided to hammer Trump after his incredible visit to storm ravaged Texas. They claimed he wasn t empathetic enough to the victims which is total bs. Real Americans aren t buying this bs line from the media because we all watched the First Couple in action. They care and we know it that s all that matters!Fox News reported:The White House announced Thursday that President Donald Trump pledged to donate $1 million in personal funds to Harvey relief efforts.Trump visited Corpus Christi, Texas, and Austin on Tuesday for briefings on Harvey s devastation. He praised first responders, telling everyone who has been affected by the storm that  we are here with you today, we are with you tomorrow and we will be with you every single day after to restore, recover and rebuild. First responders have been doing heroic work. Their courage & devotion has saved countless lives   they represent the very best of America! pic.twitter.com/I0gvCQLTKO  Donald J. Trump (@realDonaldTrump) August 31, 2017On Wednesday, Trump tweeted that on his Tuesday trip to Texas he had witnessed  first hand the horror & devastation  wrought by Harvey. He wrote that after seeing the widespread damage,  my heart goes out even more so to the great people of Texas! He plans on returning to Texas on Saturday. VP Pence and the Second Lady Karen Pence visited the storm victims today and pitched in to clear debris. '
result = predict(title,text)
print(result)

Fake News


**Saving the trained model**

In [ ]:
import pickle

In [ ]:
filename = 'trained_model_for_fake_news_detection.sav'
pickle.dump(logistic_regression_model, open(filename, 'wb'))

In [ ]:
filename = 'trained_model_for_fake_news_detection.sav'
loaded_model = pickle.load(open(filename, 'rb'))
def predict_by_loaded_model(title,text):
  text = title + ' ' + text
  text = data_preprocessing(text)
  text = tfidf.transform(countvectorizer.transform([text]))
  result = loaded_model.predict(text)[0]
  if result == 1:
    return 'Fake News'
  else:
    return 'True News'

In [ ]:
title = "President Trump Just Gave A HUGE Gift to #HurricaneHarvey Reliefâ€¦Walking the Walk!"
text =  'Yea Baby! President Trump walks the walk!President Trump just put his money where his mouth is! He donated $1 million to the Harvey Hurricane relief! What a great thing to do! He has a huge heart!Is this empathetic enough for the left? The media decided to hammer Trump after his incredible visit to storm ravaged Texas. They claimed he wasn t empathetic enough to the victims which is total bs. Real Americans aren t buying this bs line from the media because we all watched the First Couple in action. They care and we know it that s all that matters!Fox News reported:The White House announced Thursday that President Donald Trump pledged to donate $1 million in personal funds to Harvey relief efforts.Trump visited Corpus Christi, Texas, and Austin on Tuesday for briefings on Harvey s devastation. He praised first responders, telling everyone who has been affected by the storm that  we are here with you today, we are with you tomorrow and we will be with you every single day after to restore, recover and rebuild. First responders have been doing heroic work. Their courage & devotion has saved countless lives   they represent the very best of America! pic.twitter.com/I0gvCQLTKO  Donald J. Trump (@realDonaldTrump) August 31, 2017On Wednesday, Trump tweeted that on his Tuesday trip to Texas he had witnessed  first hand the horror & devastation  wrought by Harvey. He wrote that after seeing the widespread damage,  my heart goes out even more so to the great people of Texas! He plans on returning to Texas on Saturday. VP Pence and the Second Lady Karen Pence visited the storm victims today and pitched in to clear debris. '
result = predict_by_loaded_model(title,text)
print(result)

Fake News
